### Automation Script Overhaul 
For automating the parsing, transformation, and geographic file creation from .GPX<br><br>
Work taken from the script by Isias (ler_gpx.py) and workflow by Simone. Further developments by Kyle & Isais. <br><br>
Data collection must be done with Locus Map 4.x or formatted similarly to GPX files exportd by Locus Map 4 for this script to work.

### Section 1
This section takes the raw GPX files and makes them ready for QGIS and statistical analysis.

In [ ]:
import gpxpy
import pandas as pd
import geopandas as gpd
import os

gpxDict = dict()

for file in os.listdir():
    if file.endswith('.gpx'):
       gpxDict[file] = 'file_'+file
gpxDict

In [ ]:
merged = pd.DataFrame()

for i in gpxDict:
    
    # Open and read in the .gpx to a dataframe / Abra e leia no .gpx para um dataframe
    gpxCurrent = i
    gpxCurrent = open(gpxCurrent)
    gpxCurrent = gpxpy.parse(gpxCurrent)
    gpxCurrent = gpxCurrent.to_xml()
    df = pd.read_xml(gpxCurrent)

    # Ask for observer, group, climate conditions / Pergunte por observador, grupo, condições climáticas
    observer = input('Input for '+i+': Observer/Observador? ')
    group = input('Input for '+i+': Group/Grupo? (if both, mark 0) ') # or leave blank?
    weather = input('Input for '+i+': Weather conditions/Condição do clima? ')
    
    # Remove unecessary columns / Remova colunas desnecessárias
    df.pop('desc')
    df.pop('hdop')
    df.pop('time')
    df = df.drop(index=0)

    # Reorganize columns / Reorganizar colunas
    shiftPos = df.pop('name')
    df.insert(0, 'name', shiftPos)

    # Insert user input columns / Inserir colunas de entrada do usuário
    df.insert(loc=1, column='observer', value=observer, allow_duplicates=True)
    df.insert(loc=1, column='group', value=group, allow_duplicates=True)
    df.insert(loc=1, column='weather', value=weather, allow_duplicates=True)

    # Split 'name' into date, time, and observations / Dividir 'nome' em data, hora e observações
    date = df['name'].str[:10]
    df.insert(loc=0, column='date', value=date, allow_duplicates=True)

    time = df['name'].str[11:19]
    df.insert(loc=1, column='time', value=time, allow_duplicates=True)

    obs = df['name'].str[19:]
    df.insert(loc=2, column='observations', value=obs, allow_duplicates=True)

    df.pop('name')

    # Merge data frames one after another / Mesclar quadros de dados um após o outro
    merged = pd.concat([merged,df],ignore_index=True) 
    
# Convert to geodataframe for geographic use / Converter para geodataframe para uso geográfico
# If Z is wanted for the points, add / se Z for desejado para os pontos, adicione ', df.ele' ex. (df.lon, df.lat, df.ele)
gdf = gpd.GeoDataFrame(merged, geometry=gpd.points_from_xy(merged.lon, merged.lat)) 
gdf = gdf.set_crs('EPSG:4326')

gdf

In [ ]:
# # split observations into Age/Sex, Strata position, and behaviour
# ageSex = ''
# strata = ''
# behaviour = ''
# gdf.insert(loc=2, column='age/sex', value=ageSex, allow_duplicates=True)
# gdf.insert(loc=3, column='strata', value=strata, allow_duplicates=True)
# gdf.insert(loc=4, column='behaviour', value=behaviour, allow_duplicates=True)

In [ ]:
# Run loop to identify age, sex, and behaviour
for i, row in gdf.iterrows():
        # asSubstring = gdf['observations'].str[:2]
        if gdf['observations'].str[:2] == 'j2':
                ageSex = 'Juvenile 2'
                strata = gdf['observations'].str[2]
                behaviour = gdf['observations'].str[3:]
                gdf['age/sex']=ageSex
                gdf['strata']=strata
                gdf['behaviour']=behaviour
                

gdf.head()

In [ ]:
gdf.head()

In [ ]:
# Attempt to detect each scan, else create column for scan number / Tente detectar cada varredura, senão crie uma coluna para o número da varredura


#### Section 2
Analysis of scans, analyze all of this for every individual scan

In [ ]:
# Find centroid of each scan (collect geometries, find centroid)


In [ ]:
# Distance of each point/animal to centroid


In [ ]:
# points to polygons for area (ha) of group spread (convex hull)


In [ ]:
# Distance between each centroid in temporal order


In [ ]:
# Subgroups/cluster analysis, find clusters on eah scan and distance from each sub-centroid to main group centroid


#### Export Data

In [ ]:
# Export data frame to gpkg for use in QGIS / Exportar quadro de dados para gpkg para uso no QGIS
gdf.to_file('gdf.gpkg', driver="GPKG")
gdf.info()

In [ ]:
gdf.plot()